##### Ingreso el TimeSeriesSplit para ver el tema de no entrenar con cosas del futuro para predecir cosas del pasado.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
#import utils.featurizer as ft
import utils.encodeador as my_encoder
import utils.featurizerV2 as ft_v2


In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
df_test = pd.read_csv("data/test.csv")

In [3]:
train = ft_v2.featurizerv2(train,"train")
test = ft_v2.featurizerv2(test,"test")

In [4]:
drop_cols = ['titulo','fecha','descripcion','direccion', 'id']

In [5]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [6]:
X,test = my_encoder.oneHotEncoder(X,test)

In [7]:
X.shape

(240000, 639)

In [8]:
value = 2

In [9]:
tss = TimeSeriesSplit(n_splits = value)

In [10]:
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [11]:
X_train.shape

(160000, 639)

In [12]:
X_test.shape

(80000, 639)

In [13]:
y_train.shape

(160000,)

In [14]:
y_test.shape

(80000,)

In [15]:
randomForest = RandomForestRegressor(max_depth = 100, random_state = 0, n_estimators = 400)

In [16]:
randomForest.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [17]:
predicion_test = randomForest.predict(X_test)

In [18]:
mean_absolute_error(y_test, predicion_test)

578332.5335384636

In [19]:
predicion = randomForest.predict(test)

In [21]:
resultado = pd.DataFrame(predicion, index=df_test.id, columns=['target'])
display(resultado.head())
resultado.to_csv("data/workshop-randomforest-v7_0.csv", header=True)

,target
id,
4941,5.617362e+06
51775,1.061195e+06
115253,2.373094e+06
299321,1.556531e+06
173570,6.252802e+05
